In [ ]:
import requests
import pandas as pd
from datetime import datetime
import os

EIA_API_KEY = os.getenv('EIA_API_KEY')

# gathers hourly MISO demand from EIA Open Data API
def get_miso_demand(start="2024-01-01", end="2025-10-01"):
    url = (
        "https://api.eia.gov/v2/electricity/rto/region-data/data/"
        f"?api_key={EIA_API_KEY}"
        f"&frequency=hourly&data[0]=value&facets[respondent][]=MISO"
        f"&start={start}&end={end}"
    )

    resp = requests.get(url)
    resp.raise_for_status()
    json_data = resp.json()

    data = json_data["response"]["data"]
    df = pd.DataFrame(data)

    time_col = None
    for col in ["period", "date", "datetime", "local_time"]:
        if col in df.columns:
            time_col = col
            break

    # convert timestamp column
    df["timestamp"] = pd.to_datetime(df[time_col], errors="coerce")

    # standardize column naming
    if "value" not in df.columns:
        raise KeyError("Missing 'value' column — check EIA response format")
    df = df.rename(columns={"value": "demand_MWh"})

    # keep relevant columns
    keep_cols = ["timestamp", "demand_MWh", "respondent", "timezone", "product"]
    df = df[[c for c in keep_cols if c in df.columns]]

    df = df.sort_values("timestamp")
    return df

df = get_miso_demand()
df.to_csv(f"../data/eia/eia_miso_demand_{datetime.now().strftime('%Y%m%d')}.csv", index=False)
